In [1]:
!pip install peft
!pip install accelerate
!pip install bitsandBytes
!pip install transformers
!pip install datasets

In [2]:
!pip install -U datasets fsspec

In [3]:
from datasets import load_dataset

ds = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [4]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
ds['train'][0]['dialogue']

"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor."

In [6]:
sample_summary=ds['train'][0]['summary']
sample_summary

"Mr. Smith's getting a check-up, and Doctor Hawkins advises him to have one every year. Hawkins'll give some information about their classes and medications to help Mr. Smith quit smoking."

In [7]:
from transformers import pipeline
pipeline=pipeline("summarization",model="facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [8]:
sample=ds['train'][0]['dialogue']
pipeline(sample, max_length=30,min_length=20)

[{'summary_text': "Dr. Hawkins sees Mr. Smith for a check-up. He finds he hasn't had one in 5 years. Smoking is the"}]

In [9]:
sample_summary

"Mr. Smith's getting a check-up, and Doctor Hawkins advises him to have one every year. Hawkins'll give some information about their classes and medications to help Mr. Smith quit smoking."

In [10]:

max_input = 512
max_target = 128
batch_size = 3
model_checkpoints = "facebook/bart-large-xsum"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [12]:

def preprocess_data(data_to_process):
  #get all the dialogues
  inputs = [dialogue for dialogue in data_to_process['dialogue']]
  #tokenize the dialogues
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)

  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [13]:
tokenize_data = ds.map(preprocess_data, batched = True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [14]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [15]:
from transformers import TrainingArguments, Trainer
args=TrainingArguments(
    output_dir='/content',
    per_device_train_batch_size=8,
    num_train_epochs=2,
    remove_unused_columns=True




)

In [16]:
trainer=Trainer(
    model=model,
    args=args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['test']
)

In [17]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohitnilvarn2004 (mohitnilvarn2004-trinity-academy-of-engineering-pune) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.465200
1000,0.331100
1500,0.315700
2000,0.233400
2500,0.217100
3000,0.211000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3116, training_loss=0.2926974461840726, metrics={'train_runtime': 6877.547, 'train_samples_per_second': 3.623, 'train_steps_per_second': 0.453, 'total_flos': 2.700212334231552e+16, 'train_loss': 0.2926974461840726, 'epoch': 2.0})

In [18]:
eval= trainer.evaluate()

In [19]:
eval

{'eval_loss': 0.32369664311408997,
 'eval_runtime': 109.7179,
 'eval_samples_per_second': 13.671,
 'eval_steps_per_second': 1.713,
 'epoch': 2.0}

In [20]:
model.save_pretrained('/content/bart_directory')
tokenizer.save_pretrained("/content/bart_directory")

('/content/bart_directory/tokenizer_config.json',
 '/content/bart_directory/special_tokens_map.json',
 '/content/bart_directory/vocab.json',
 '/content/bart_directory/merges.txt',
 '/content/bart_directory/added_tokens.json',
 '/content/bart_directory/tokenizer.json')

In [21]:
tokenizer = AutoTokenizer.from_pretrained('/content/bart_directory')
model = AutoModelForSeq2SeqLM.from_pretrained('/content/bart_directory')

def summarize(text):
  input=tokenizer.encode(text, max_length=1024, truncation=True, return_tensors='pt')
  output=model.generate(input, max_length=150, min_length=40, early_stopping=True)
  return tokenizer.decode(output[0], skip_special_tokens=True)

In [22]:
summarize('''The increase in global temperatures has led to more frequent and severe weather events, posing a significant threat to ecosystems and human societies.
 One of the major impacts of climate change is the rise in sea levels, which results from the melting of polar ice caps and glaciers.
 Coastal areas are particularly vulnerable, as they face higher risks of flooding, storm surges, and erosion.
 Additionally, the warming atmosphere can hold more moisture, leading to intense and unpredictable precipitation patterns.
 This variability can cause both severe droughts and devastating floods, affecting agricultural productivity and water resources.
 The effects of climate change are widespread, influencing not only the environment but also the socio-economic stability of communities.
 For example, changing weather patterns can disrupt food supply chains, increase the prevalence of diseases, and force people to migrate from their homes.
 To mitigate these effects, countries are investing in adaptive infrastructure, developing early warning systems, and implementing policies to reduce greenhouse gas emissions.
 The collaboration between governments, scientists, and communities is crucial to building resilience against the adverse impacts of climate change.
 By taking proactive measures, societies can better prepare for the challenges posed by a changing climate and work towards a more sustainable future.
''')

'The increase in global temperatures has led to more frequent and severe weather events, posing a significant threat to ecosystems and human societies. One of the major impacts of climate change is the rise in sea levels. The warming atmosphere can hold more moisture, causing severe droughts and devastating floods, affecting agricultural productivity and water resources. To mitigate these effects, countries are investing in adaptive infrastructure, developing early warning systems, and implementing policies to reduce greenhouse gas emissions.'